In [10]:
import pandas as pd

#Load the CSV
data = pd.read_csv('LFB Purchase Orders £5k to £10k FY 2024-25 Q1.csv')

# Display the first few rows
print(data.head())

     Order Number  Order Date  Net Amount  Line Number  \
0  TSR/18271       10/05/2024      9156.0          1.0   
1  TSR/18271       10/05/2024      9156.0          2.0   
2  TSR/18271       10/05/2024      9156.0          3.0   
3  TSR/18621       15/05/2024      5306.4          1.0   
4  TSF/08046       28/06/2024      7700.0          1.0   

                                    Part Description Unit Cost Order Quantity  \
0  DRESSING, CHITOGAUZE          (IEC PACK)      ...     23.17            250   
1  DRESSING, CORIUS SEAL         (IEC PACK)      ...      5.91            350   
2  DRESSING, TRAUMA, PROMETHEUS  (IEC PACK)      ...      3.70            350   
3  WATER, BOTTLED, DRINKING,      OPERATIONAL USE...      7.37            720   
4  PROFESSIONAL SERVICES, FINANCE CONSULTANCY    ...  7,700.00              1   

   Line Cost  Vendor ID                     Vendor Name  \
0     5792.5    20169.0  Safeguard Medical Technologies   
1     2068.5    20169.0  Safeguard Medical Tec

In [8]:
#Summary of the dataset (columns, types, null values)
print(data.info())

#Descriptive statistics for numerical columns
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order Number      302 non-null    object 
 1   Order Date        302 non-null    object 
 2   Net Amount        302 non-null    float64
 3   Line Number       302 non-null    float64
 4   Part Description  302 non-null    object 
 5   Unit Cost         302 non-null    object 
 6   Order Quantity    302 non-null    object 
 7   Line Cost         302 non-null    float64
 8   Vendor ID         302 non-null    float64
 9   Vendor Name       302 non-null    object 
 10  Location Name     302 non-null    object 
dtypes: float64(4), object(7)
memory usage: 26.2+ KB
None
         Net Amount  Line Number     Line Cost     Vendor ID
count    302.000000   302.000000    302.000000    302.000000
mean    7397.469272    17.225166   2990.316623  18362.000000
std     1246.651823    27.104859   3304.138

In [11]:
#Data Cleaning 
# Convert 'Order Date' to datetime
data['Order Date'] = pd.to_datetime(data['Order Date'], errors='coerce')

# Convert 'Unit Cost' and 'Order Quantity' to numeric, removing commas
data['Unit Cost'] = pd.to_numeric(data['Unit Cost'].str.replace(',', ''), errors='coerce')
data['Order Quantity'] = pd.to_numeric(data['Order Quantity'].str.replace(',', ''), errors='coerce')


In [12]:
#Handling missing values 
# Check for missing values
print(data.isnull().sum())

# Fill missing values (median) or drop rows
data['Unit Cost'].fillna(data['Unit Cost'].median(), inplace=True)
data.dropna(subset=['Order Date', 'Net Amount', 'Vendor Name'], inplace=True)


Order Number          1
Order Date          129
Net Amount            1
Line Number           1
Part Description      1
Unit Cost             1
Order Quantity        1
Line Cost             1
Vendor ID             1
Vendor Name           1
Location Name         1
dtype: int64


C:\Users\mjlri\AppData\Local\Temp\ipykernel_33204\1172379638.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Unit Cost'].fillna(data['Unit Cost'].median(), inplace=True)


In [14]:
#Create a New Feature Total Line Cost: Featuring Engineering
# Create a new column for total line cost
data['Total Line Cost'] = data['Unit Cost'] * data['Order Quantity']

In [15]:
# Extract month and year from 'Order Date': allow trend analysis in Power BI like visualising  monthly spending patterns
data['Order Month'] = data['Order Date'].dt.month
data['Order Year'] = data['Order Date'].dt.year

In [16]:
#Key Insights via Data Aggregation
#Total Expenditure by vendor: help identify the top vendors in terms of spending
vendor_spend = data.groupby('Vendor Name')['Net Amount'].sum().reset_index()
vendor_spend = vendor_spend.sort_values(by='Net Amount', ascending=False)
print(vendor_spend.head(10))  # Top 10 vendors by expenditure

                       Vendor Name  Net Amount
11  CODE BLUE                        743203.33
7   BRISSCO SIGNS & GRAPHICS          72968.00
25  NPOWER LTD                        60595.60
24  Mitie FM Limited,                 60490.14
20  INFICON                           32800.00
34  Telefonica UK Ltd                 29005.81
32  Safeguard Medical Technologies    27468.00
9   Bellrock (Portal) Property        23481.45
33  THERMO FISHER SCIENTIFIC          21631.60
22  IRON MOUNTAIN UK PLC              20023.66


In [17]:
#Monthly Trends: total expenditure per month to observe trends over time helpful in Power Bi
monthly_spend = data.groupby(['Order Year', 'Order Month'])['Net Amount'].sum().reset_index()
print(monthly_spend)

    Order Year  Order Month  Net Amount
0         2024            1    11808.32
1         2024            2    72157.10
2         2024            3    95757.94
3         2024            4    47059.13
4         2024            5    64251.82
5         2024            6    31869.56
6         2024            7   781230.27
7         2024            8    88626.34
8         2024            9    22722.21
9         2024           10    57221.46
10        2024           11     5743.86
11        2024           12    26895.99


In [18]:
#Top products by Spend: most expensive product lines in terms of total expenditure 
product_spend = data.groupby('Part Description')['Total Line Cost'].sum().reset_index()
product_spend = product_spend.sort_values(by='Total Line Cost', ascending=False)
print(product_spend.head(10))  # Top 10 products by total line cost

                                     Part Description  Total Line Cost
39  MOBILE SERVICES               (ICT)           ...         29005.81
45  PPM TASK                                      ...         22881.39
58  TECHNOLOGY SOLUTIONS          (ICT)           ...         20023.66
59  TRAINING COURSES - EXTERNAL   (SUBJECTIVE: 166...         18000.00
60  TRAINING UNITARY PAYMENT -    BABCOCKS        ...         11236.53
36  LEE GREEN FIRE STATION - ELECTRICAL REWIRE PRO...          8824.00
27  HYDRANT PLATE, 6 SLOT, 4 HOLE,100 AT TOP & 1 A...          8260.00
2   ATTENDANCE                                    ...          7730.20
38  MERTON LOC- DRYING ROOM WORKS- BUDGET 2132    ...          7457.00
50  R1 DRY SUITS SAF10025 RED/BLK AS PER AGREED SP...          7425.00


In [19]:
#Detecting Outliers in Net Amount or Line Cost
# Identify outliers by looking at high or low values for Net Amount
outliers = data[(data['Net Amount'] > data['Net Amount'].quantile(0.95)) | (data['Net Amount'] < data['Net Amount'].quantile(0.05))]
print(outliers)

       Order Number Order Date  Net Amount  Line Number  \
7    TSF/07011      2024-08-04     9808.34          1.0   
8    TSF/07351      2024-01-05     5121.91          1.0   
9    TSF/07432      2024-09-05     9624.21          1.0   
13   TSF/07825      2024-10-06     9573.26          1.0   
26   TSF/07791      2024-06-06     5402.71          1.0   
40   TSF/06972      2024-04-04     5000.00          1.0   
76   QZK/00015      2024-03-06     9668.10          1.0   
77   QZK/00015      2024-03-06     9668.10          2.0   
78   QZK/00015      2024-03-06     9668.10          3.0   
79   QZK/00015      2024-03-06     9668.10          4.0   
80   QZK/00015      2024-03-06     9668.10          5.0   
186  TSF/07794      2024-07-06     5135.86          1.0   
191  TSF/07461      2024-10-05     5250.00          1.0   
265  TSF/07078      2024-12-04     5230.96          1.0   
274  WLV/00003      2024-03-05     5000.00          2.0   
275  WLV/00003      2024-03-05     5000.00          3.0 

In [ ]:
# Save the cleaned and transformed data
data.to_csv('cleaned_purchase_orders.csv', index=False)

In [21]:
#Trend Analysis: Monthly Expenditure over time: expenditure fluctuation
# Grouping by year and month to get total expenditure per month
monthly_trend = data.groupby(['Order Year', 'Order Month'])['Net Amount'].sum().reset_index()

# Sort by Year and Month for proper visualization
monthly_trend = monthly_trend.sort_values(by=['Order Year', 'Order Month'])

print(monthly_trend)

    Order Year  Order Month  Net Amount
0         2024            1    11808.32
1         2024            2    72157.10
2         2024            3    95757.94
3         2024            4    47059.13
4         2024            5    64251.82
5         2024            6    31869.56
6         2024            7   781230.27
7         2024            8    88626.34
8         2024            9    22722.21
9         2024           10    57221.46
10        2024           11     5743.86
11        2024           12    26895.99


In [22]:
#Vendor Performance: Top Vendors by total spend: identify most orders or highest total expenditure
# Group by Vendor to calculate total expenditure
top_vendors = data.groupby('Vendor Name')['Net Amount'].sum().reset_index()

# Sort in descending order to get the top vendors
top_vendors = top_vendors.sort_values(by='Net Amount', ascending=False)

# Display the top 10 vendors
print(top_vendors.head(10))




                       Vendor Name  Net Amount
11  CODE BLUE                        743203.33
7   BRISSCO SIGNS & GRAPHICS          72968.00
25  NPOWER LTD                        60595.60
24  Mitie FM Limited,                 60490.14
20  INFICON                           32800.00
34  Telefonica UK Ltd                 29005.81
32  Safeguard Medical Technologies    27468.00
9   Bellrock (Portal) Property        23481.45
33  THERMO FISHER SCIENTIFIC          21631.60
22  IRON MOUNTAIN UK PLC              20023.66


In [23]:
#Cost Efficiency: Unit cost per vendor/product
# Group by Vendor Name and calculate the average Unit Cost
avg_unit_cost_per_vendor = data.groupby('Vendor Name')['Unit Cost'].mean().reset_index()

# Sort by Unit Cost to identify vendors with the highest/lowest average costs
avg_unit_cost_per_vendor = avg_unit_cost_per_vendor.sort_values(by='Unit Cost', ascending=False)

# Display the vendors with the highest average unit cost
print(avg_unit_cost_per_vendor.head(10))

                       Vendor Name    Unit Cost
34  Telefonica UK Ltd               9668.603333
27  Premier People Solutions Ltd    9000.000000
0   ABM Technical Solutions Ltd     7370.640000
22  IRON MOUNTAIN UK PLC            6674.553333
31  SSS Public Safety Limited       6267.530000
2   ANSCO MUSIC CLUB LTD            6000.000000
28  Royal United Services Inst.     5981.500000
6   BABCOCK TRAINING LTD            5618.265000
26  PHOENIX SOFTWARE LTD            5580.000000
30  SPECIALIST COMPUTER CENTRE      5402.710000


In [24]:
# Group by Part Description to calculate total order quantities
high_volume_products = data.groupby('Part Description')['Order Quantity'].sum().reset_index()

# Sort to find products ordered in the largest quantities
high_volume_products = high_volume_products.sort_values(by='Order Quantity', ascending=False)

# Display the top 10 high-volume products
print(high_volume_products.head(10))


                                     Part Description  Order Quantity
36  LEE GREEN FIRE STATION - ELECTRICAL REWIRE PRO...          8824.0
38  MERTON LOC- DRYING ROOM WORKS- BUDGET 2132    ...          7457.0
1   ACCIDENT REPAIR PAYMENTS      - PFI CONTRACTOR...          7200.0
8   DISPLAY ENERGY CERTIFICATES                   ...          7156.0
6   CROYDON SUPPLY` AND DELIVERY OF PELLETS FOR BI...          7000.0
44  PPE PAYMENTS - BRISTOL UNIFORMS LTD           ...          6835.0
16  GP TECH-SERV- ANNUAL PPM BLANKET ORDER- BUDGET...          6686.0
37  MERTON LOC- CHANGING ROOM LOCKER ROOMS        ...          6025.0
46  PRIVACY FOR ALL                               ...          5744.0
43  POSTAL/COURIER SERVICES (HQ)  - CITY SPRINT   ...          5000.0


In [25]:
#Outliers: identify anomalies in expenditure
# Calculate the IQR for 'Net Amount' to detect outliers: Interquartile method 
Q1 = data['Net Amount'].quantile(0.25)
Q3 = data['Net Amount'].quantile(0.75)
IQR = Q3 - Q1

In [26]:
# Define thresholds for detecting outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Find outliers based on the thresholds
outliers = data[(data['Net Amount'] < lower_bound) | (data['Net Amount'] > upper_bound)]

# Display outliers
print(outliers)

       Order Number Order Date  Net Amount  Line Number  \
0    TSR/18271      2024-10-05     9156.00          1.0   
1    TSR/18271      2024-10-05     9156.00          2.0   
2    TSR/18271      2024-10-05     9156.00          3.0   
7    TSF/07011      2024-08-04     9808.34          1.0   
8    TSF/07351      2024-01-05     5121.91          1.0   
..              ...        ...         ...          ...   
287  PGS/13736      2024-04-06     7080.00          2.0   
288  PGS/13741      2024-07-06     7440.00          1.0   
289  PGS/13741      2024-07-06     7440.00          2.0   
296  PGS/13679      2024-05-04     8200.00          3.0   
297  PGS/13679      2024-05-04     8200.00          4.0   

                                      Part Description  Unit Cost  \
0    DRESSING, CHITOGAUZE          (IEC PACK)      ...      23.17   
1    DRESSING, CORIUS SEAL         (IEC PACK)      ...       5.91   
2    DRESSING, TRAUMA, PROMETHEUS  (IEC PACK)      ...       3.70   
7    MOBILE SER

In [28]:
# Save the cleaned and analyzed dataset to CSV files for Power BI
monthly_trend.to_csv('monthly_trend.csv', index=False)
top_vendors.to_csv('top_vendors.csv', index=False)
avg_unit_cost_per_vendor.to_csv('avg_unit_cost_per_vendor.csv', index=False)
high_volume_products.to_csv('high_volume_products.csv', index=False)
outliers.to_csv('expenditure_outliers.csv', index=False)
